In [ ]:
! pwd

In [ ]:
from smartprint import smartprint as sprint
import numpy as np

# speed = np.load("nina_mean_gt_pred_unc_error_speed.npy")
# volume = np.load("nina_mean_gt_pred_unc_error_volume.npy")

speed = np.load("mean_gt_pred_unc_err_antwerp_tta0_speed.npy")
volume = np.load("mean_gt_pred_unc_err_antwerp_tta0_volume.npy")

sprint (speed.shape, volume.shape)

In [ ]:
speed_gt, speed_pred, speed_unc, speed_err = speed[0, :, :, :], speed[1, :, :, :], \
                                            speed[2, :, :, :], speed[3, :, :, :]
volume_gt, volume_pred, volume_unc, volume_err = volume[0, :, :, :], volume[1, :, :, :], \
                                            volume[2, :, :, :], volume[3, :, :, :]

In [ ]:
non_zero_speed = speed_gt > 0

density_gt = volume_gt/speed_gt
non_zero_density = density_gt > 0
non_outlier_densities = density_gt < 255

valid_indices = non_zero_density & non_zero_speed & non_outlier_densities

sprint ("\n", speed_gt.shape, non_zero_speed.shape, valid_indices.shape, non_zero_density.shape)
speed_gt = speed_gt[valid_indices].tolist()
speed_pred = speed_pred[valid_indices].tolist()
speed_unc = speed_unc[valid_indices].tolist()
speed_err = speed_err[valid_indices].tolist()

volume_gt = volume_gt[valid_indices].tolist()
volume_pred = volume_pred[valid_indices].tolist()
volume_unc = volume_unc[valid_indices].tolist()
volume_err = volume_err[valid_indices].tolist()

density_gt = density_gt[valid_indices].tolist()



In [ ]:
len(density_gt)

In [ ]:
def sort_two_lists_according_to_first(X,Y):
    ss = sorted(zip(X, Y))
    X = [x for x, y in ss]
    Y = [y for x, y in ss]    
    return X, Y


import matplotlib.pyplot as plt
import numpy as np


smoothing_window = 1001


X, _ = sort_two_lists_according_to_first(density_gt, speed_gt)
plt.plot(X[smoothing_window//2:-smoothing_window//2], label="density")    

names = ['speed_gt', 'speed_err', 'speed_unc', 'volume_err', 'volume_gt', 'volume_unc']
for index, var in enumerate([speed_gt, speed_err, speed_unc, volume_err, volume_gt, volume_unc]):
    
    sprint (len(var))
    _, Y = sort_two_lists_according_to_first(density_gt, var)
    Y = Y[smoothing_window//2:-smoothing_window//2]
    
    plt.plot(np.convolve(Y, [1/smoothing_window]*smoothing_window, "same"), label=names[index])
#     plt.show()
plt.yscale("log")
p = np.array([0.0, 25.0, 50.0, 75.0, 100.0])
plt.xticks(((len(X)-1) * p/100), map(str, p))
plt.legend()
plt.show()




############ Same thing with absolute values of quantiles on the xticks
def sort_two_lists_according_to_first(X,Y):
    ss = sorted(zip(X, Y))
    X = [x for x, y in ss]
    Y = [y for x, y in ss]    
    return X, Y


import matplotlib.pyplot as plt
import numpy as np



X, _ = sort_two_lists_according_to_first(density_gt, speed_gt)
plt.plot(X[smoothing_window//2:-smoothing_window//2], label="density")    

names = ['speed_gt', 'speed_err', 'speed_unc', 'volume_err', 'volume_gt', 'volume_unc']
for index, var in enumerate([speed_gt, speed_err, speed_unc, volume_err, volume_gt, volume_unc]):
    
    sprint (len(var))
    _, Y = sort_two_lists_according_to_first(density_gt, var)
    Y = Y[smoothing_window//2:-smoothing_window//2]
    
    plt.plot(np.convolve(Y, [1/smoothing_window]*smoothing_window, "same"), label=names[index])
#     plt.show()
plt.yscale("log")

pos = np.array([0.0, 25.0, 50.0, 75.0, 99.0])
perc = np.round(np.percentile(X, p), 2)

plt.xticks(((len(X)-1) * pos/100), map(str, perc))
plt.legend()
plt.show()



In [ ]:
l = len(X)
plt.hist(X[:int(l*0.99)], 200)
plt.show()

In [ ]:
np.percentile(X, p)

In [ ]:
# speed density plots
def sort_three_lists_according_to_first(X,Y,Z):
    ss = sorted(zip(X, Y, Z))
    X = [x for x,y,z in ss]
    Y = [y for x,y,z in ss]
    Z = [z for x,y,z in ss]
    return X, Y, Z

def conv(X):
    return np.convolve(X, [1/smoothing_window]*smoothing_window)
                       
import matplotlib.cm as cm


U, D, S = sort_three_lists_according_to_first(conv(speed_unc), conv(density_gt), conv(speed_gt))
sprint (len(U), len(S), len(D))
max_99 = 0.99 * np.max(U)
c = cm.viridis(np.array(U)/ max_99)
c = c.tolist()

plt.scatter(D, S, color=c, s=(np.array(U)), alpha=np.clip(np.array(U)/ max_99, 0,1))
# plt.yscale("log")
# plt.xscale("log")
# plt.xlim(0,0.2)
# plt.ylim(0,20)
plt.colorbar()
plt.xlabel("Density-Proxy")
plt.ylabel("Speed-GT")
plt.title("Speed vs Density curve colored by Speed-Uncertainty")
plt.show()

In [ ]:
# speed density plots
def sort_three_lists_according_to_first(X,Y,Z):
    ss = sorted(zip(X, Y, Z))
    X = [x for x,y,z in ss]
    Y = [y for x,y,z in ss]
    Z = [z for x,y,z in ss]
    return X, Y, Z

def conv(X):
    return np.convolve(X, [1/smoothing_window]*smoothing_window)
                       
import matplotlib.cm as cm


U, D, V = sort_three_lists_according_to_first(conv(volume_unc), conv(density_gt), conv(volume_gt))
sprint (len(U), len(S), len(D))

max_99 = 0.99 * np.max(U)
c = cm.viridis(np.array(U)/ max_99)
c = c.tolist()

plt.scatter(D, V, color=c, s=1, alpha=0.5) # np.clip(np.array(U)/ max_99, 0,1))
# plt.yscale("log")
# plt.xscale("log")
# plt.xlim(0,0.2)
# plt.ylim(0,20)
plt.colorbar()
plt.xlabel("Density-Proxy")
plt.ylabel("Volume-GT")
plt.title("Volume vs Density curve colored by Volume-Uncertainty")
plt.show()

In [ ]:
plt.hist(U, 209)
plt.xlim(0,6)
plt.title("V")
plt.show()

In [ ]:
U[-1]

In [ ]:
cm.GnBu